# Coffeeshop Location Recommender for Philadelphia

## Imports

In [9]:
import json
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

## Load Data

In [10]:
projected_crs = 'EPSG:6564'

# Load park data
park_gdf = gpd.read_file('/kaggle/input/philadelphia-data/PPR_Properties.geojson')
park_gdf = park_gdf.to_crs(projected_crs) # Project in plain coordinate system

# Load university data
university_gdf = gpd.read_file('/kaggle/input/philadelphia-data/Universities_Colleges.geojson')
university_gdf = university_gdf.to_crs(projected_crs) # Project in plain coordinate system

# Load university data
streets_gdf = gpd.read_file('/kaggle/input/philadelphia-data/CompleteStreets.geojson')
streets_gdf = streets_gdf.to_crs(projected_crs) # Project in plain coordinate system

## Data Analysis Functions

### Check Coordinate Validity

In [11]:
def is_in_philadelphia(lat, lon):
    """Check if coordinates are in philadelphia"""
    # TODO
    pass

def is_in_park(lat, lon):
    """Check if coordinates are in a park"""
    # TODO
    pass

def is_in_water(lat, lon):
    """Check if coordinates are in the water, i.e. river or lake"""
    # TODO
    pass

### Calculate Distances


In [12]:
def distance_to(coordinates_df, gdf, column_name):
    """Calculate the distance from given point to the closest polygon in gdf.
    
    Returns: Pandas DataFrame with two columns.
    """
    # Create geopandas DataFrame from given points
    point_gdf = gpd.points_from_xy(coordinates_df["Longitude"], coordinates_df["Latitude"], crs="WGS84")
    # Transform coordinates to planar
    point_gdf = point_gdf.to_crs(projected_crs)

    min_distance_list = []

    for point in point_gdf:
        # Calculate distance of point to all polygons
        distances = gdf['geometry'].distance(point)
        # Find index of closest park
        idxmin = distances.idxmin()

        min_distance_list.append([distances[idxmin], gdf[column_name].loc[idxmin]])
    
    min_distances_df = pd.DataFrame(min_distance_list, columns=["Distance [m]", column_name])
    return min_distances_df

def distance_to_park(coordinates_df):
    """Calculate the distance to closest parks.
    
    Returns: Pandas DataFrame with two columns (Distance [m], PUBLIC_NAME)
    """
    return distance_to(coordinates_df, park_gdf, 'PUBLIC_NAME')

def distance_to_university(coordinates_df):
    """Calculate the distance to closest university.
    
    Returns: Pandas DataFrame with two columns (Distance [m], NAME)
    """
    return distance_to(coordinates_df, university_gdf, 'NAME')


def distance_to_public_transport(lat, lon, n=1):
    """Calculate the distance to n closest bus/train stations
    Returns: List of n Tuples with distance and the object that is close by."""
    # TODO
    pass

## Recommender

### Retrieve Data

In [13]:
def retrieve_data(coordinates):
    """Calculate the data categories for the given coordinates.
    
    Returns: DataFrame with data categories as columns.
    """
    data_df = coordinates

    # Distance to university
    data_df['University Distance'] = distance_to_university(coordinates)["Distance [m]"]

    # Distance to park
    data_df['Park Distance'] = distance_to_park(coordinates)["Distance [m]"]
    return data_df

In [73]:
def linear_score(data_series, min_val=0, max_val=500, lower_is_better=True):
    """Project values between min_val and max_val linearly between 1 and 0."""

    data_series[data_series < min_val] = 0
    data_series[data_series > max_val] = max_val

    if lower_is_better:
        return 1 - data_series/max_val

    return data_series/max_val

def conv_data_to_scores(data_df, weights):
    """Convert the data values into scores between 0 and 1, where 1 is a good score.
    
    Returns: DataFrame with the same column names as data_df but scores instead of raw data values.
    """

    score_df = data_df[['Latitude', 'Longitude']].copy(deep=True)
        
    if "Park Distance" in weights.keys():
        score_df["Park Distance"] = linear_score(data_df["Park Distance"], min_val = 10, max_val=500, lower_is_better=True)
        
    if "University Distance" in weights.keys():
        score_df["University Distance"] = linear_score(data_df["University Distance"], min_val = 10, max_val=500, lower_is_better=True)
    
    return score_df
        

In [74]:
def combine_scores(score_df, weights):
    return score_df[weights.keys()].mul(weights).sum(1)

In [75]:
def calculate_score(coordinates, weights={}):
    
    data_df = retrieve_data(coordinates)

    if len(weights) == 0:
        # Create a weight dictionary with equal weighting for each data column
        data_columns = data_df.drop(["Latitude", "Longitude"], axis=1).columns
        weights = {col_name: 1/len(data_columns) for col_name in data_columns}

    score_df_ = conv_data_to_scores(data_df, weights)
    score_df_['Score'] = combine_scores(score_df_, weights)
    return score_df_

## Recommend

In [105]:
def create_coordinate_grid(top_left_lat, top_left_lon, hor_dist=1000, hor_res=500, ver_dist=1000, ver_res=500):
    """Create a coordinate grip.
    
    Approximations: 1 degree of latitude corresponds to 111km and 1 degree of longitude corresponds to 73km."""
    x = np.arange(0, hor_dist, hor_res)
    y = np.arange(0, ver_dist, ver_res)
    lon_grid, lat_grid = np.meshgrid(x, y)
    lon_grid = lon_grid * 1/73000
    lat_grid = lat_grid * 1/111000
    lon_grid = top_left_lon + lon_grid
    lat_grid = top_left_lat - lat_grid
    return pd.DataFrame({'Latitude': lat_grid.flatten(), 'Longitude':lon_grid.flatten()})

In [111]:
# Coordinates for which the score is to be calculated
#coordinates = pd.DataFrame([[40.051854, -75.047369]], columns=["Latitude", "Longitude"])
coordinates = create_coordinate_grid(
    top_left_lat=39.963097,
    top_left_lon=-75.185065,
    hor_dist=3000, hor_res=50,
    ver_dist=3000, ver_res=50
)
score_df = calculate_score(coordinates)
score_df

,Latitude,Longitude,Park Distance,University Distance,Score
0,39.963097,-75.185065,0.723605,0.561948,0.642776
1,39.963097,-75.184380,0.756734,0.456898,0.606816
2,39.963097,-75.183695,0.740071,0.347751,0.543911
3,39.963097,-75.183010,0.799552,0.236264,0.517908
4,39.963097,-75.182325,0.895976,0.123327,0.509652
...,...,...,...,...,...
3595,39.936520,-75.147394,0.872721,0.000000,0.436361
3596,39.936520,-75.146709,1.000000,0.000000,0.500000
3597,39.936520,-75.146024,0.965424,0.000000,0.482712
3598,39.936520,-75.145339,0.942777,0.000000,0.471389


## Visualizations

### Score Heatmap

In [112]:
import folium

from folium.plugins import HeatMap

heat_map = folium.Map([40, -75], zoom_start=10)

heat_coords = score_df[['Latitude', 'Longitude', 'Score']]
folium.plugins.HeatMap(heat_coords).add_to(heat_map)

heat_map

----------------------------------------------------------------------------------------------------
## Playground

In [113]:
# Create a Shapely Point object
lat = 40.051854
lon = -75.047369

point_gdf = gpd.GeoSeries([Point(lon, lat)])
point_gdf.crs = "WGS84"
point_gdf = point_gdf.to_crs(projected_crs)

dist = park_gdf.distance(point_gdf[0])
dist

# Calculate distances to all polygons
park_gdf['distance_to_point'] = park_gdf['geometry'].distance(point_gdf[0])

# Sort by distance
park_gdf = park_gdf.sort_values(by='distance_to_point', ascending=True)

# Select the n closest polygons
n = 3
closest_polygons = park_gdf.head(n)
closest_polygons

,OBJECTID,PUBLIC_NAME,PARENT_NAME,NESTED,OFFICIAL_NAME,LABEL,ALIAS,DPP_ASSET_ID,ADDRESS_911,ZIP_CODE,...,COUNCIL_DISTRICT,POLICE_DISTRICT,CITY_SCALE_MAPS,LOCAL_SCALE_MAPS,PROGRAM_SITES,COMMENTS,Shape__Area,Shape__Length,geometry,distance_to_point
7,8,Pennypack Park,Pennypack Park,N,Pennypack Park,Pennypack Park,Little City (Rhawn/Holmhurst); Crystal Spring ...,1651,NaN,19152,...,6;10,7;8;15,Y,Y,N,NaN,9.356229e+06,50426.819252,"MULTIPOLYGON (((828559.869 87496.124, 828561.4...",389.071410
130,131,Bradford Park,Bradford Park,N,NaN,Bradford,Rhawnhurst,236,2400 FAUNCE ST,19152,...,10,2,Y,Y,N,Named for William Bradford in 1982.,5.886321e+04,995.670354,"POLYGON ((830106.334 83134.999, 830101.608 829...",498.176094
105,106,George C Pelbano Playground,George C Pelbano Playground,N,George C. Pelbano Playground,Pelbano,Rhawnhurst Raiders Athletic Association;The Fr...,1987,2138 SOLLY AVE,19152,...,10,7,Y,Y,Y,Site has Northeast Older Adult Center. Footbal...,7.721695e+04,1183.509334,"POLYGON ((830367.836 84215.389, 830364.443 842...",928.879563


In [ ]:

pd.set_option('display.max_columns', 500)
streets_gdf

In [ ]:
university_gdf.plot()

In [83]:
# Opening JSON file
f = open('/kaggle/input/philadelphia-data/PPR_Properties.geojson')
geo_json_data = json.load(f)
m = folium.Map([40, -75], zoom_start=10)

folium.GeoJson(geo_json_data).add_to(m)

m

In [85]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from geopy.distance import great_circle

# Define your parameters in meters
top_left_latitude = 40.0  # Top left latitude in degrees
top_left_longitude = -120.0  # Top left longitude in degrees
horizontal_dimension_meters = 10000.0  # Horizontal dimension in meters
vertical_dimension_meters = 5000.0  # Vertical dimension in meters
horizontal_point_distance_meters = 1000.0  # Horizontal point distance in meters
vertical_point_distance_meters = 1000.0  # Vertical point distance in meters

# Convert meters to degrees using geopy
origin = Point(top_left_longitude, top_left_latitude)
horizontal_dimension_degrees = great_circle(kilometers=horizontal_dimension_meters / 1000).destination(origin, 90).longitude - top_left_longitude
vertical_dimension_degrees = great_circle(kilometers=vertical_dimension_meters / 1000).destination(origin, 0).latitude - top_left_latitude

# Create a grid of coordinates using NumPy matrix multiplication
rows = int(vertical_dimension_degrees / vertical_point_distance_meters)
cols = int(horizontal_dimension_degrees / horizontal_point_distance_meters)

lats = np.linspace(top_left_latitude, top_left_latitude + vertical_dimension_degrees, rows + 1)
lons = np.linspace(top_left_longitude, top_left_longitude + horizontal_dimension_degrees, cols + 1)

# Perform matrix multiplication to generate grid coordinates
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Flatten the latitude and longitude grids
latitude = lat_grid.flatten()
longitude = lon_grid.flatten()

# Create a GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(longitude, latitude)]
gdf = gpd.GeoDataFrame({'geometry': geometry}, crs='EPSG:4326')

# Create a DataFrame with latitude and longitude columns
df = gdf[['geometry']].copy()
df['latitude'] = latitude
df['longitude'] = longitude

# Display the resulting DataFrame
print(df[['latitude', 'longitude']])



TypeError: Failed to create Point instance from <shapely.geometry.point.Point object at 0x7c5e8a9dac20>.

In [87]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import pyproj

# Define your parameters in meters
top_left_latitude = 40.0  # Top left latitude in degrees
top_left_longitude = -120.0  # Top left longitude in degrees
horizontal_dimension_meters = 10000.0  # Horizontal dimension in meters
vertical_dimension_meters = 5000.0  # Vertical dimension in meters
horizontal_point_distance_meters = 1000.0  # Horizontal point distance in meters
vertical_point_distance_meters = 1000.0  # Vertical point distance in meters

# Create a projection transformer for meters to degrees
project_meters_to_degrees = pyproj.Transformer.from_crs(3857, 4326, always_xy=True)

# Convert the top-left point from degrees to meters
top_left_x, top_left_y = project_meters_to_degrees.transform(top_left_longitude, top_left_latitude)

# Create a mesh grid in meters
rows = int(vertical_dimension_meters / vertical_point_distance_meters)
cols = int(horizontal_dimension_meters / horizontal_point_distance_meters)

lats_meters = np.linspace(top_left_y, top_left_y + vertical_dimension_meters, rows + 1)
lons_meters = np.linspace(top_left_x, top_left_x + horizontal_dimension_meters, cols + 1)

# Convert the mesh grid back to degrees
lats_degrees, lons_degrees = project_meters_to_degrees.transform(lons_meters, lats_meters)

# Create a GeoDataFrame with the mesh grid in degrees
geometry = [Point(lon, lat) for lat, lon in zip(lats_degrees, lons_degrees)]
gdf_degrees = gpd.GeoDataFrame({'geometry': geometry}, crs='EPSG:4326')

# Extract latitude and longitude from the geometry column
gdf_degrees['latitude'] = gdf_degrees['geometry'].apply(lambda point: point.y)
gdf_degrees['longitude'] = gdf_degrees['geometry'].apply(lambda point: point.x)

# Display the resulting DataFrame
print(gdf_degrees[['latitude', 'longitude']])


ProjError: x, y, z, and time must be same size if included.